# Developing AI Systems with the OpenAI API - Part 2

## Function Calling

Dive into the mechanics of function calling, where you'll master defining parameters to call specific functions, extracting structured data, and working with multiple functions. Learn to seamlessly integrate external APIs and manipulate responses to communicate with other systems, to develop enhanced AI system interactions.

In [ ]:
import os

# Set your OpenAI API key
openai_api_key = os.environ['OPENAI_API_KEY']

### Using the tools parameter
You are developing an AI application for a real estate agency and have been asked to extract some key data from listings: house type, location, price, number of bedrooms. Use the Chat Completions endpoint with function calling to extract the information.

The message_listing message, containing the real estate listing, and function_definition, containing the function to call defined as a tool to be passed to the model, have been preloaded.

In [2]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

message_listing = [
    {'role': 'system',
     'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {'role': 'user',
     'content': 'Step into this beautiful two-story, single-family home located in Springfield, USA, priced at $350,000. This charming property features 4 bedrooms, 2.5 bathrooms, a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a large backyard perfect for family gatherings. The master bedroom includes an en-suite bathroom and a walk-in closet. Enjoy the convenience of an attached two-car garage and a recently updated HVAC system. Located near top-rated schools, parks, and shopping centers, this home is ideal for families looking for comfort and convenience.'}
]

function_definition = [
    {'type': 'function',
     'function': {'name': 'real_estate_info',
                  'description': 'Get the information about homes for sale from the body of the input text',
                  'parameters': {'type': 'object',
                                 'properties': {'home type': {'type': 'string', 
                                                              'description': 'Home type'},
                                                'location': {'type': 'string', 
                                                             'description': 'Location'},
                                                'price': {'type': 'integer', 
                                                          'description': 'Price'},
                                                'bedrooms': {'type': 'integer', 
                                                             'description': 'Number of bedrooms'}}}}}
]

response= client.chat.completions.create(
    model="gpt-4o-mini",
    # Add the message
    messages=message_listing,
    # Add your function definition
    tools=function_definition
)

# Print the response
print(response.choices[0].message.tool_calls[0].function.arguments)

{"home type":"single-family","location":"Springfield, USA","price":350000,"bedrooms":4}


### Building a function dictionary
You are working on a research project where you need to extract key information from a collection of scientific research papers. The goal is to create a summary of key information from the papers you are given, that includes the title and year of publication. To compile this, you decide to use the OpenAI API with function calling to extract the key information.

The get_response() function and messages, containing the text of the research paper, have been preloaded. The function_definition variable has also partially been filled already.

In [3]:
client = OpenAI(api_key=openai_api_key)

function_definition = [
    {'type': 'function',
     'function': {'name': 'extract_review_info',
                  'description': 'Extract the title and year of publication from research papers.',
                  'parameters': {}}}
]

# Define the function parameter type
function_definition[0]['function']['parameters']['type'] = 'object'

# Define the function properties
function_definition[0]['function']['parameters']['properties'] = {
    'title': { 
        'type': 'string', 
        'description': 'Research paper title'
    },
    'year': {
        'type': 'string',
        'description': 'Year of paper publication'
    }
}

messages = [
    {'role': 'system',
     'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {'role': 'user',
     'content': '\nA. M. Turing (1950) Computing Machinery and Intelligence. Mind 49: 433-460.\nCOMPUTING MACHINERY AND INTELLIGENCE\nBy A. M. Turing\n1. The Imitation Game\nI propose to consider the question, "Can machines think?" This should begin with\ndefinitions of the meaning of the terms "machine" and "think." The definitions might be\nframed so as to reflect so far as possible the normal use of the words, but this attitude is\ndangerous, If the meaning of the words "machine" and "think" are to be found by\nexamining how they are commonly used it is difficult to escape the conclusion that the\nmeaning and the answer to the question, "Can machines think?" is to be sought in a\nstatistical survey such as a Gallup poll. But this is absurd. Instead of attempting such a\ndefinition I shall replace the question by another, which is closely related to it and is\nexpressed in relatively unambiguous words.\nThe new form of the problem can be described in terms of a game which we call the\n\'imitation game." It is played with three people, a man (A), a woman (B), and an\ninterrogator (C) who may be of either sex. The interrogator stays in a room apart front the\nother two. The object of the game for the interrogator is to determine which of the other\ntwo is the man and which is the woman. He knows them by labels X and Y, and at the\nend of the game he says either "X is A and Y is B" or "X is B and Y is A." The\ninterrogator is allowed to put questions to A and B thus:\nC: Will X please tell me the length of his or her hair?\nNow suppose X is actually A, then A must answer. It is A\'s object in the game to try and\ncause C to make the wrong identification. His answer might therefore be:\n"My hair is shingled, and the longest strands are about nine inches long."\nIn order that tones of voice may not help the interrogator the answers should be written,\nor better still, typewritten. The ideal arrangement is to have a teleprinter communicating\nbetween the two rooms. Alternatively the question and answers can be repeated by an\nintermediary. The object of the game for the third player (B) is to help the interrogator.\nThe best strategy for her is probably to give truthful answers. She can add such things as\n"I am the woman, don\'t listen to him!" to her answers, but it will avail nothing as the man\ncan make similar remarks.\nWe now ask the question, "What will happen when a machine takes the part of A in this\ngame?" Will the interrogator decide wrongly as often when the game is played like this as\nhe does when the game is played between a man and a woman? These questions replace\nour original, "Can machines think?\n                 '}
]

def get_response(messages, function_definition):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=function_definition
    )
    return response.choices[0].message.tool_calls[0].function.arguments

response = get_response(messages, function_definition)
print(response)

{"title":"Computing Machinery and Intelligence","year":"1950"}


### Extracting the response
You work for a company that has just launched a new smartphone. The marketing team has collected customer reviews from various online platforms and wants to analyze the feedback to understand the customer sentiment and the most talked-about features of the smartphone. To accelerate this, you've used the OpenAI API to extract structured data from these reviews, using function calling. You now need to write a function to clean the output and return a dictionary of the response from the function only.

The get_response() function, messages variable (containing the review) and function_definition (containing the function to extract sentiment and product features from reviews) have been preloaded. Notice that both messages and function_definition can be passed as arguments to the get_response() function to get the response from the chat completions endpoint.

In [4]:
client = OpenAI(api_key=openai_api_key)

messages = [
    {'role': 'system',
     'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {'role': 'user',
     'content': "\nI recently purchased the TechCorp ProMax and I'm absolutely in love with its powerful processor. However, I think they could really improve the product by deciding to offer more color options.\n"}
]

function_definition = [
    {'type': 'function',
     'function': {'name': 'extract_sentiment_and_product_features',
                  'description': 'Extract sentiment and product features from reviews',
                  'parameters': {'type': 'object',
                                 'properties': {'product': {'type': 'string',
                                                'description': 'The product name'},
                                 'sentiment': {'type': 'string',
                                               'description': 'The overall sentiment of the review'},
                                 'features': {'type': 'array',
                                              'items': {'type': 'string'},
                                              'description': 'List of features mentioned in the review'},
                                 'suggestions': {'type': 'array',
                                                 'items': {'type': 'string'},
                                                 'description': 'Suggestions for improvement'}}}}}
]

def get_response(messages, function_definition):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=function_definition
    )
    return response

response = get_response(messages, function_definition)

# Define the function to extract the data dictionary
def extract_dictionary(response):
  return response.choices[0].message.tool_calls[0].function.arguments

# Print the data dictionary
print(extract_dictionary(response))

{"product": "TechCorp ProMax", "sentiment": "positive", "features": ["powerful processor"], "suggestions": ["offer more color options"]}


### Parallel function calling
After extracting the data from customers' reviews for the marketing team, the company you're working for asks you if there's a way to generate a response to the customer that they can post on their review platform. You decide to use parallel function calling to apply both functions and generate data as well as the responses. You use a function named reply_to_review and ask to return the review reply as a reply property.

In this exercise, the get_response() function, messages and function_definition variable have been preloaded. The messages already contain the user's review, and function_definition contains the function asking to extract structured data.

In [5]:
client = OpenAI(api_key=openai_api_key)

messages = [
    {'role': 'system', 
     'content': 'Apply both functions and return responses.'},
    {'role': 'user',
     'content': "\nI recently purchased the TechCorp ProMax and I'm absolutely in love with its powerful processor. However, I think they could really improve the product by deciding to offer more color options.\n"}
]

function_definition = [
    {'type': 'function',
     'function': {'name': 'extract_sentiment_and_product_features',
                  'description': 'Extract sentiment and product features from reviews',
                  'parameters': {'type': 'object',
                                 'properties': {'product': {'type': 'string',
                                                'description': 'The product name'},
                                 'sentiment': {'type': 'string',
                                               'description': 'The overall sentiment of the review'},
                                 'features': {'type': 'array',
                                              'items': {'type': 'string'},
                                              'description': 'List of features mentioned in the review'},
                                 'suggestions': {'type': 'array',
                                                 'items': {'type': 'string'},
                                                 'description': 'Suggestions for improvement'}}}}}
]

# Append the second function
function_definition.append(
    {'type': 'function', 
     'function':{'name': 'reply_to_review', 
                 'description': 'Get the review reply', 
                 'parameters': {'type': 'object', 
                                'properties': {'reply': {'type':'string', 
                                                         'description':'The review reply'}}}}}
)

def get_response(messages, function_definition):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=function_definition
    )
    return str(response.choices[0].message.tool_calls[0].function.arguments) + ",\n " + str(response.choices[0].message.tool_calls[1].function.arguments)

response = get_response(messages, function_definition)

# Print the response
print(response)

{"product": "TechCorp ProMax", "sentiment": "positive", "features": ["powerful processor"], "suggestions": ["more color options"]},
 {"reply": "Thank you for your feedback! We're thrilled to hear that you love the powerful processor of the TechCorp ProMax. Your suggestion for more color options has been noted and we appreciate your input!"}


### Setting a specific function
You have been given a few customer reviews to analyze, and have been asked to extract for each one the product name, variant, and customer sentiment. To ensure that the model extracts this specific information, you decide to use function calling and specify the function for the model to use. Use the Chat Completions endpoint with function calling and tool_choice to extract the information.

In this exercise, the messages and function_definition have been preloaded.

In [6]:
client = OpenAI(api_key=openai_api_key)

model = 'gpt-4o-mini'

messages = [
    {'role': 'system',
     'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. The sentiment should be positive or negative or neutral."},
    {'role': 'user',
     'content': "\nI recently purchased the steel color version of the thermal mug and I am absolutely thrilled with it! The mug keeps my drinks hot for hours, which is perfect for my long commutes. The steel color gives it a sleek and professional look that I love. Overall, I'm very happy with my purchase and would highly recommend this product to anyone in need of a reliable and stylish thermal mug.\n                 "}
]

function_definition = [
    {'type': 'function',
     'function': {'name': 'extract_review_info',
                  'description': 'Get the information about products and customer sentiment from the body of the input text',
                  'parameters': {'type': 'object',
                                 'properties': {'product name': {'type': 'string',
                                                                 'description': 'Home type'},
                                                'product variant': {'type': 'string', 
                                                                    'description': 'Location'},
                                                'sentiment': {'type': 'string', 
                                                              'description': 'Price'}}}}}
]

response= client.chat.completions.create(
    model=model,
    messages=messages,
    # Add the function definition
    tools=function_definition,
    # Specify the function to be called for the response
    tool_choice={'type': 'function',
                 'function': {'name': 'extract_review_info'}}
)

# Print the response
print(response.choices[0].message.tool_calls[0].function.arguments)

{"product name":"thermal mug","product variant":"steel color","sentiment":"positive"}


### Avoiding inconsistent responses
The team you were working with on the previous project is enthusiastic about the reply generator and asks you if more reviews can be processed. However, some reviews have been mixed up with other documents, and you're being asked not to return responses if the text doesn't contain a review, or relevant information. For example, the review you're considering now doesn't contain a product name, and so there should be no product name being returned.

In this exercise, the get_response() function, and messages and function_definition variables have been preloaded. The messages already contain the user's review, and function_definition contains the two functions: one asking to extract structured data, and one asking to generate a reply.

In [7]:
client = OpenAI(api_key=openai_api_key)

messages = [
    {'role': 'system', 
     'content': 'Apply both functions and return responses.'},
    {'role': 'user',
     'content': '\nThrilled with the quality, but I think it should come with a wider choice of screen sizes.\n'}
]

# Modify the messages
messages.append({'role': 'system', 
                 'content': "Don't make assumptions about what values to plug into functions."})

function_definition = [
    {'type': 'function',
     'function': {'name': 'extract_sentiment_and_product_features',
                  'description': 'Extract sentiment and product features from reviews',
                  'parameters': {'type': 'object',
                                 'properties': {'product': {'type': 'string',
                                                'description': 'The product name'},
                                 'sentiment': {'type': 'string',
                                               'description': 'The overall sentiment of the review'},
                                 'features': {'type': 'array',
                                              'items': {'type': 'string'},
                                              'description': 'List of features mentioned in the review'},
                                 'suggestions': {'type': 'array',
                                                 'items': {'type': 'string'},
                                                 'description': 'Suggestions for improvement'}}}}},
    {'type': 'function', 
     'function':{'name': 'reply_to_review', 
                 'description': 'Reply politely to the customer who wrote the review', 
                 'parameters': {'type': 'object', 
                                'properties': {'reply': {'type':'string', 
                                                         'description':'Reply to post in response to the review'}}}}}
]

def get_response(messages, function_definition):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=function_definition
    )
    return str(response.choices[0].message.tool_calls[0].function.arguments) + ",\n " + str(response.choices[0].message.tool_calls[1].function.arguments)

response = get_response(messages, function_definition)

print(response)

{"sentiment": "positive", "features": ["quality", "screen sizes"], "suggestions": ["wider choice of screen sizes"]},
 {"reply": "Thank you for your feedback! We're thrilled to hear you're happy with the quality. We appreciate your suggestion for more screen size options and will take it into consideration."}


### Defining a function with external APIs
You are developing a flight simulation application and have been asked to develop a system that provides specific information about airports mentioned in users' requests. You decide to use the OpenAI API to convert the user request into airport codes, and then call the AviationAPI to return the information requested. As the first step in your coding project, you configure the function to pass to the tools parameter in the Chat Completions endpoint.

In this exercise, the get_airport_info() and get_response() functions have been preloaded. The get_airport_info() function uses the AviationAPI and takes as input one airport code, returning the response with the requested airport information.

In [8]:
import requests
import json

client = OpenAI(api_key=openai_api_key)

# Define the function to pass to tools
function_definition = [
    {"type": 'function',
     'function' : {"name": 'get_airport_info',
                   'description': 'The function uses the Aviation API and takes as input one airport code, returning the response with the requested airport information',
                   'parameters': {"type": 'object', 
                                  'properties': {"airport_code": {'type': 'string',
                                                                  'description': 'The airport code to be passed to the get_airport_info function'}}},
                   "result": {'type': 'string'}}}
]

def get_airport_info(airport_code):
    url = "https://api.aviationapi.com/v1/airports"
    querystring = {"apt":airport_code}
    response = requests.request("GET", url, params=querystring)
    return response.text

def postprocess_response(response):
    if response.choices[0].finish_reason=='tool_calls':
        function_call = response.choices[0].message.tool_calls[0].function
        if function_call.name == "get_airport_info":
            code = json.loads(function_call.arguments)["airport_code"]
            airport_info = get_airport_info(code)
            if airport_info:  
                return airport_info
            else:
                print("Apologies, I couldn't make any recommendations based on the request.")
        else:
            print("Apologies, I couldn't find any airport.")
    else:
        print("I am sorry, but I could not understand your request.")

def get_response(function_definition):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": "You are an AI assistant, an aviation specialist. You should interpret the user prompt, and based on it extract an airport code corresponding to their message."},
            {"role": "user", 
             "content": "I'm planning to land a plane in JFK airport in New York and would like to have the corresponding information."}
        ],
        tools=function_definition,
    )
    return postprocess_response(response)

response = get_response(function_definition)
print(response)

{"JFK":[{"site_number":"15793.*A","type":"AIRPORT","facility_name":"JOHN F KENNEDY INTL","faa_ident":"JFK","icao_ident":"KJFK","region":"AEA","district_office":"NYC","state":"NY","state_full":"NEW YORK","county":"QUEENS","city":"NEW YORK","ownership":"PU","use":"PU","manager":"CHARLES EVERETT","manager_phone":"(718) 244-3501","latitude":"40-38-23.7400N","latitude_sec":"146303.7400N","longitude":"073-46-43.2930W","longitude_sec":"265603.2930W","elevation":"13","magnetic_variation":"13W","tpa":"","vfr_sectional":"NEW YORK","boundary_artcc":"ZNY","boundary_artcc_name":"NEW YORK","responsible_artcc":"ZNY","responsible_artcc_name":"NEW YORK","fss_phone_number":"","fss_phone_numer_tollfree":"1-800-WX-BRIEF","notam_facility_ident":"JFK","status":"O","certification_typedate":"I E S 05\/1973","customs_airport_of_entry":"N","military_joint_use":"N","military_landing":"Y","lighting_schedule":"","beacon_schedule":"SS-SR","control_tower":"Y","unicom":"122.950","ctaf":"","effective_date":"11\/04\/20

### Calling an external API
Now that you have a clearly structured function definition, you move on to improving your endpoint request. You use the Chat Completions endpoint and pass a system message to ensure that the AI assistant is aware that it is in the aviation space and that it needs to extract the corresponding airport code based on the user input.

In this exercise, the get_airport_info() function has been preloaded. The get_airport_info() function uses the AviationAPI and takes as input one airport code, returning the response with the requested airport information. The print_response() function has also been preloaded to print the output.

In [9]:
client = OpenAI(api_key=openai_api_key)

function_definition = [
    {"type": 'function',
     'function' : {"name": 'get_airport_info',
                   'description': 'This function calls the Aviation API to return the airport code corresponding to the airport in the request',
                   'parameters': {"type": 'object', 
                                  'properties': {"airport_code": {'type': 'string',
                                                                  'description': 'The code to be passed to the get_airport_info function.'}}},
                   "result": {'type': 'string'}}}
]

# Call the Chat Completions endpoint 
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {'role': 'system', 'content': 'You are an AI assistant, a specialist in the aviation space. You need to interpret the user prompt, and based on it extract the corresponding airport code based.'},
        {"role": 'user', "content": "I'm planning to land a plane in JFK airport in New York and would like to have the corresponding information."}
    ],
    tools=function_definition)

def print_response(response):
    print(postprocess_response(response))

print_response(response)

{"JFK":[{"site_number":"15793.*A","type":"AIRPORT","facility_name":"JOHN F KENNEDY INTL","faa_ident":"JFK","icao_ident":"KJFK","region":"AEA","district_office":"NYC","state":"NY","state_full":"NEW YORK","county":"QUEENS","city":"NEW YORK","ownership":"PU","use":"PU","manager":"CHARLES EVERETT","manager_phone":"(718) 244-3501","latitude":"40-38-23.7400N","latitude_sec":"146303.7400N","longitude":"073-46-43.2930W","longitude_sec":"265603.2930W","elevation":"13","magnetic_variation":"13W","tpa":"","vfr_sectional":"NEW YORK","boundary_artcc":"ZNY","boundary_artcc_name":"NEW YORK","responsible_artcc":"ZNY","responsible_artcc_name":"NEW YORK","fss_phone_number":"","fss_phone_numer_tollfree":"1-800-WX-BRIEF","notam_facility_ident":"JFK","status":"O","certification_typedate":"I E S 05\/1973","customs_airport_of_entry":"N","military_joint_use":"N","military_landing":"Y","lighting_schedule":"","beacon_schedule":"SS-SR","control_tower":"Y","unicom":"122.950","ctaf":"","effective_date":"11\/04\/20

### Handling the response with external API calls
To better connect your flight simulation application to other systems, you'd like to add some checks to make sure that the model has found an appropriate answer. First you check that the response has been produced via tool_calls. If that is the case, you check that the function used to produce the result was get_airport_info. If so, you load the airport code extracted from the user's prompt, and call the get_airport_info() function with the code as argument. Finally, if that produces a response, you return the response.

In this exercise, the response, the json library, and get_airport_info() function have been preloaded.

In [11]:
import json

def get_airport_info(airport_code):
    url = "https://api.aviationapi.com/v1/airports"
    querystring = {"apt":airport_code}
    response = requests.request("GET", url, params=querystring)
    return response.text

# Check that the response has been produced using function calling
if response.choices[0].finish_reason=='tool_calls':
# Extract the function
    function_call = response.choices[0].message.tool_calls[0].function
    print(function_call)

    # Check function name
    if function_call.name == "get_airport_info":
        # Extract airport code
        code = json.loads(function_call.arguments)["airport_code"]
        airport_info = get_airport_info(code)
        print(airport_info)
    else:
        print("Apologies, I couldn't find any airport.")
else:
    print("I am sorry, but I could not understand your request.")

Function(arguments='{"airport_code":"JFK"}', name='get_airport_info')
{"JFK":[{"site_number":"15793.*A","type":"AIRPORT","facility_name":"JOHN F KENNEDY INTL","faa_ident":"JFK","icao_ident":"KJFK","region":"AEA","district_office":"NYC","state":"NY","state_full":"NEW YORK","county":"QUEENS","city":"NEW YORK","ownership":"PU","use":"PU","manager":"CHARLES EVERETT","manager_phone":"(718) 244-3501","latitude":"40-38-23.7400N","latitude_sec":"146303.7400N","longitude":"073-46-43.2930W","longitude_sec":"265603.2930W","elevation":"13","magnetic_variation":"13W","tpa":"","vfr_sectional":"NEW YORK","boundary_artcc":"ZNY","boundary_artcc_name":"NEW YORK","responsible_artcc":"ZNY","responsible_artcc_name":"NEW YORK","fss_phone_number":"","fss_phone_numer_tollfree":"1-800-WX-BRIEF","notam_facility_ident":"JFK","status":"O","certification_typedate":"I E S 05\/1973","customs_airport_of_entry":"N","military_joint_use":"N","military_landing":"Y","lighting_schedule":"","beacon_schedule":"SS-SR","contro